## ⚠️ ORDEN DE EJECUCIÓN DE CELDAS

**Si acabas de reiniciar el kernel, sigue este orden:**

1. **CELDA 1**: Importar librerías y configurar directorio
2. **CELDA 2**: Cargar datos (brechas.csv, proyectos.csv)
3. **CELDA 3**: Generar embeddings de brechas
4. **CELDA 4**: Crear índice FAISS
5. **CELDA 5**: Preparar etiquetas multi-label
6. **CELDA 6**: Crear dataset de PyTorch
7. **CELDA 7**: ⚡ **ENTRENAR MODELO** (10-60 min según GPU)
8. **CELDA 8**: Clasificación híbrida (requiere modelo entrenado)
9. **CELDA 9**: Integración LLM (opcional)

**IMPORTANTE:** No puedes ejecutar CELDA 8 sin antes completar CELDA 7.

---

## 📚 Recursos y Documentación de Modelos

### Modelos utilizados en este notebook:

#### 🔹 **BETO (BERT Español) - Clasificación**
- **Repositorio:** [dccuchile/bert-base-spanish-wwm-cased](https://huggingface.co/dccuchile/bert-base-spanish-wwm-cased)
- **Tipo:** Modelo BERT pre-entrenado en español con Whole Word Masking
- **Uso:** Clasificación multi-label de proyectos en brechas

#### 🔹 **MPNet - Embeddings Semánticos**
- **Repositorio:** [paraphrase-multilingual-mpnet-base-v2](https://huggingface.co/sentence-transformers/paraphrase-multilingual-mpnet-base-v2)
- **Tipo:** Modelo de embeddings multilingüe (768 dimensiones)
- **Uso:** Búsqueda por similitud semántica de brechas

#### 🔹 **Alternativas de Modelos de Embeddings:**
- [MiniLM](https://huggingface.co/sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2) (384 dims, más rápido)
- [Modelo Español específico](https://huggingface.co/hiiamsid/sentence_similarity_spanish_es) (768 dims)

---

### 📖 Documentación General:

- **HuggingFace Transformers:** https://huggingface.co/docs/transformers
- **Sentence-Transformers:** https://www.sbert.net/
- **FAISS (búsqueda vectorial):** https://github.com/facebookresearch/faiss/wiki
- **PyTorch:** https://pytorch.org/docs/

### 🔍 Buscar más modelos:

- **Hub de modelos:** https://huggingface.co/models
  - Filtrar por idioma: `Spanish` o `Multilingual`
  - Filtrar por tarea: `Text Classification`, `Sentence Similarity`
- **Leaderboard MTEB** (comparar embeddings): https://huggingface.co/spaces/mteb/leaderboard


# Pipeline Híbrido de Clasificación de Brechas

## Requisitos para Desarrollo Local

### Software Base:
- **Python 3.9+** (recomendado 3.10 o 3.11)
- **Git** para control de versiones
- **VS Code** con extensión de Jupyter

### Archivos de Datos Requeridos:
```
project/
├── data/
│   ├── brechas.csv          # Columnas: id, brecha
│   └── proyectos.csv        # Columnas: project_id, title, description, brecha_ids
├── models/                  # Se creará durante el entrenamiento
├── outputs/                 # Se creará automáticamente
└── notebooks/
    └── hybrid_pipeline.ipynb
```

### Hardware Recomendado (Local):
- **RAM**: Mínimo 8GB, recomendado 16GB
- **GPU** (opcional): NVIDIA con 6GB+ VRAM para acelerar entrenamiento
- **Disco**: 5GB+ libres para modelos y datasets

---

## Despliegue en Google Cloud

Este notebook será ejecutado en **Google Cloud Platform** con:
- **Vertex AI Workbench** o **AI Platform Notebooks**
- **Google Cloud Storage** para datos y modelos
- **GPU**: Tesla T4 o V100 (configuración en cloud)
- **Docker containers** para producción

Por ahora, instala los paquetes básicos localmente para desarrollo y testing.

In [1]:
# ========================================
# OPCIÓN 2: Para VS CODE (Jupyter local)
# ========================================

# IMPORTANTE: Tu sistema tiene CUDA 12.6, instalar PyTorch compatible
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

# Luego instalar el resto de paquetes (incluyendo accelerate para Trainer)
%pip install -U pip
%pip install sentence-transformers faiss-cpu transformers accelerate datasets scikit-learn pandas numpy tqdm

# ⚠️ DESPUÉS DE EJECUTAR: REINICIA EL KERNEL del notebook (botón "Restart" arriba)


Looking in indexes: https://download.pytorch.org/whl/cu121
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## 📦 Instalación de Dependencias

**Nota:** La sintaxis cambia según el entorno:
- **VS Code/Jupyter**: usa `%pip install`
- **Google Colab**: usa `!pip install`

Ejecuta solo UNA de las siguientes celdas según tu entorno.

In [2]:
# ========================================
# OPCIÓN 1: Para GOOGLE COLAB
# ========================================
# Descomenta (quita el #) si estás en Google Colab:

# !pip install -U pip
# !pip install sentence-transformers faiss-cpu transformers accelerate datasets scikit-learn pandas numpy tqdm


In [2]:
# CELDA 1
# ====================================================================
# IMPORTACIÓN DE LIBRERÍAS Y CONFIGURACIÓN DEL DIRECTORIO DE TRABAJO
# ====================================================================

# os: Módulo del sistema operativo para operaciones de archivos y directorios
import os

# pandas: Librería para manipulación y análisis de datos en estructuras tabulares (DataFrames)
import pandas as pd

# numpy: Librería fundamental para computación científica, manejo de arrays y operaciones matemáticas
import numpy as np

# tqdm: Librería para mostrar barras de progreso en bucles e iteraciones
from tqdm import tqdm

# faiss: Librería de Facebook AI para búsqueda de similitud vectorial eficiente (k-NN aproximado)
import faiss

# torch: Framework de deep learning PyTorch para redes neuronales y cómputo tensorial
import torch

# MultiLabelBinarizer: Transforma etiquetas multi-clase en formato binario (one-hot encoding)
from sklearn.preprocessing import MultiLabelBinarizer

# SentenceTransformer: Modelo para generar embeddings semánticos de texto (vectores densos)
from sentence_transformers import SentenceTransformer

# Transformers de HuggingFace:
# - AutoTokenizer: Tokenizador automático que carga el apropiado según el modelo
# - AutoModelForSequenceClassification: Modelo pre-entrenado para clasificación de secuencias
# - Trainer: API de alto nivel para entrenar modelos de transformers
# - TrainingArguments: Configuración de hiperparámetros para el entrenamiento
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

# Path: Clase para manejo de rutas de archivos de forma multiplataforma
from pathlib import Path

# ====================================================================
# DETECCIÓN Y CONFIGURACIÓN DEL DIRECTORIO RAÍZ DEL PROYECTO
# ====================================================================

# Obtener el directorio de trabajo actual donde se ejecuta el notebook
CWD = Path.cwd()

# Verificar si existe la carpeta 'data' en el directorio actual
if not (CWD / 'data').exists():
    # Si no existe, buscar en el directorio padre
    parent = CWD.parent
    
    # Verificar si el directorio padre contiene 'data'
    if (parent / 'data').exists():
        # Cambiar el directorio de trabajo a la raíz del proyecto
        os.chdir(parent)
        print(f"[INFO] Cambiado directorio de trabajo a raíz del proyecto: {parent}")
    else:
        # Advertir si no se encuentra 'data' en ninguna ubicación esperada
        print(f"[ADVERTENCIA] No se encontró carpeta 'data' en {CWD} ni en {parent}. Las lecturas fallarán si las rutas no existen.")
else:
    # El directorio actual ya es la raíz del proyecto
    print(f"[INFO] Directorio de trabajo ya está en raíz del proyecto: {CWD}")


c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[INFO] Cambiado directorio de trabajo a raíz del proyecto: d:\UPC\SEPTIMO CICLO\MODULO REGULAR\Proyecto de Investigación 1\TRABAJO\TP1\Modelo\project


## 🔍 Verificación de GPU/VRAM

Ejecuta la siguiente celda para verificar si tus modelos están usando la GPU (VRAM) o solo la RAM del sistema.

In [3]:
import torch

print("="*60)
print("VERIFICACIÓN DE HARDWARE")
print("="*60)

# 1. Verificar disponibilidad de CUDA
cuda_available = torch.cuda.is_available()
print(f"\n✓ CUDA disponible: {cuda_available}")

if cuda_available:
    # 2. Información de la GPU
    print(f"✓ GPU detectada: {torch.cuda.get_device_name(0)}")
    print(f"✓ Versión CUDA: {torch.version.cuda}")
    
    # 3. Memoria VRAM
    total_vram = torch.cuda.get_device_properties(0).total_memory / 1024**3
    vram_allocated = torch.cuda.memory_allocated(0) / 1024**3
    vram_reserved = torch.cuda.memory_reserved(0) / 1024**3
    
    print(f"\n📊 MEMORIA GPU (VRAM):")
    print(f"  - Total VRAM: {total_vram:.2f} GB")
    print(f"  - VRAM en uso: {vram_allocated:.2f} GB")
    print(f"  - VRAM reservada: {vram_reserved:.2f} GB")
    print(f"  - VRAM disponible: {total_vram - vram_reserved:.2f} GB")
    
    # 4. Prueba de tensor en GPU
    print(f"\n🧪 Prueba de GPU:")
    test_tensor = torch.randn(1000, 1000).cuda()
    print(f"  - Tensor creado en: {test_tensor.device}")
    print(f"  - ✓ GPU está funcionando correctamente")
    
    # Limpiar memoria
    del test_tensor
    torch.cuda.empty_cache()
else:
    print("\n⚠️ GPU NO DISPONIBLE")
    print("Los modelos se ejecutarán en CPU (usando RAM del sistema)")
    print("\nPara usar GPU:")
    print("1. Instala PyTorch con CUDA:")
    print("   %pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118")
    print("2. Verifica que tienes drivers NVIDIA actualizados")

print("="*60)

VERIFICACIÓN DE HARDWARE

✓ CUDA disponible: True
✓ GPU detectada: NVIDIA GeForce RTX 3060
✓ Versión CUDA: 12.1

📊 MEMORIA GPU (VRAM):
  - Total VRAM: 12.00 GB
  - VRAM en uso: 0.00 GB
  - VRAM reservada: 0.00 GB
  - VRAM disponible: 12.00 GB

🧪 Prueba de GPU:
  - Tensor creado en: cuda:0
  - ✓ GPU está funcionando correctamente


In [4]:
# Verificar versión exacta de PyTorch instalada
import torch
print(f"PyTorch versión: {torch.__version__}")
print(f"CUDA build version: {torch.version.cuda}")
print(f"cuDNN version: {torch.backends.cudnn.version()}")
print(f"CUDA disponible: {torch.cuda.is_available()}")

PyTorch versión: 2.5.1+cu121
CUDA build version: 12.1
cuDNN version: 90100
CUDA disponible: True


In [5]:
# CELDA 2
# data/brechas.csv => columns: id,brecha
# data/proyectos.csv => columns: project_id,title,description,brecha_ids (e.g. "1,3")
from pathlib import Path
PROJECT_ROOT = Path.cwd()  # después del ajuste en CELDA 1 debe ser la raíz
brechas_path = PROJECT_ROOT / 'data' / 'brechas.csv'
proyectos_path = PROJECT_ROOT / 'data' / 'proyectos.csv'

if not brechas_path.exists():
    raise FileNotFoundError(f"No se encontró {brechas_path}. Verifica que el archivo exista y que el notebook se ejecute desde la carpeta 'project' o que la celda 1 haya cambiado el cwd correctamente.")
if not proyectos_path.exists():
    raise FileNotFoundError(f"No se encontró {proyectos_path}. Verifica que el archivo exista.")

brechas = pd.read_csv(brechas_path)
proyectos = pd.read_csv(proyectos_path)

# Quick preview
print("Brechas:", len(brechas))
print("Proyectos etiquetados:", len(proyectos))
display(brechas.head())
display(proyectos.head())


Brechas: 48
Proyectos etiquetados: 817


,id,brecha
0,1,INDICADOR DE BRECHA POR DEFINIR
1,2,PORCENTAJE DE ALIMENTOS AGROPECUARIOS DE PRODU...
2,3,PORCENTAJE DE CAPITALES DE DISTRITO QUE NO CUE...
3,4,PORCENTAJE DE CEMENTERIOS CON CAPACIDAD INSTAL...
4,5,PORCENTAJE DE CENTROS CUNA MÁS EN CONDICIONES ...


,project_id,title,description,brecha_ids
0,1,CONSTRUCCION DE VEREDAS Y RECONSTRUCCION DE PI...,CONSTRUCCION DE VEREDAS Y RECONSTRUCCION DE PI...,18
1,2,MEJORAMIENTO DE VEREDAS EN LA AVENIDA PUYA RAY...,MEJORAMIENTO DE VEREDAS EN LA AVENIDA PUYA RAY...,18
2,3,MEJORAMIENTO DE LA INFRAESTRUCTURA VIAL Y PEAT...,MEJORAMIENTO DE LA INFRAESTRUCTURA VIAL Y PEAT...,18
3,4,MEJORAMIENTO DE AV. JORGE BASADRE GROHMANN DES...,MEJORAMIENTO DE AV. JORGE BASADRE GROHMANN DES...,18
4,5,FORTALECIMIENTO DEL SERVICIO DE LIMPIEZA PUBLI...,FORTALECIMIENTO DEL SERVICIO DE LIMPIEZA PUBLI...,36


In [6]:
# CELDA 3 - GENERACIÓN DE EMBEDDINGS SEMÁNTICOS DE LAS BRECHAS
# ====================================================================
# En esta celda se genera la representación vectorial (embeddings) de cada brecha
# para permitir búsqueda por similitud semántica
# ====================================================================

# ====================================================================
# OPCIONES DE MODELOS DE EMBEDDINGS (elige UNO según tus recursos)
# ====================================================================

# OPCIÓN 1: MiniLM (RÁPIDO, 384 dims) - Recomendado para desarrollo/testing
# Ventaja: Bajo consumo de RAM/VRAM, velocidad alta
# Desventaja: Menor precisión semántica
# embed_model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

# OPCIÓN 2: MPNet (BALANCEADO, 768 dims) - Recomendado para producción
# Ventaja: Mejor balance precisión/velocidad
# Desventaja: Requiere ~2x más RAM que MiniLM
embed_model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-mpnet-base-v2")

# OPCIÓN 3: Modelos específicos para español (768 dims)
# Ventaja: Entrenados específicamente en español, mejor comprensión del idioma
# Desventaja: Mayor consumo de recursos
# embed_model = SentenceTransformer("hiiamsid/sentence_similarity_spanish_es")
# O alternativa:
# embed_model = SentenceTransformer("hackathon-pln-es/paraphrase-spanish-distilroberta")

# OPCIÓN 4: Modelo GRANDE (1024 dims) - Solo si tienes GPU potente
# Ventaja: Máxima precisión semántica
# Desventaja: Requiere GPU con 8GB+ VRAM, muy lento en CPU
# embed_model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-mpnet-base-v2")

# ====================================================================
# COMPARACIÓN DE MODELOS
# ====================================================================
# | Modelo                                  | Dims | Velocidad | Precisión | RAM    |
# |-----------------------------------------|------|-----------|-----------|--------|
# | paraphrase-multilingual-MiniLM-L12-v2   | 384  | ⚡⚡⚡      | ⭐⭐      | ~500MB |
# | paraphrase-multilingual-mpnet-base-v2   | 768  | ⚡⚡        | ⭐⭐⭐    | ~1GB   |
# | hiiamsid/sentence_similarity_spanish_es | 768  | ⚡⚡        | ⭐⭐⭐⭐  | ~1GB   |
# | LaBSE (Google)                          | 768  | ⚡         | ⭐⭐⭐⭐  | ~2GB   |
# ====================================================================

# ====================================================================
# CODIFICACIÓN DE BRECHAS A VECTORES
# ====================================================================

# Extraer la columna "brecha" del DataFrame como lista de strings
texts = brechas["brecha"].tolist()

# Generar embeddings para cada brecha
# - show_progress_bar=True: muestra barra de progreso durante la codificación
# - convert_to_numpy=True: convierte resultado a array numpy para compatibilidad con FAISS
# - normalize_embeddings=True: normaliza vectores a norma L2=1 (facilita cálculo de similitud coseno)
brecha_embeddings = embed_model.encode(
    texts, 
    show_progress_bar=True, 
    convert_to_numpy=True, 
    normalize_embeddings=True
)

# ====================================================================
# GUARDAR EMBEDDINGS Y DATOS PARA USO POSTERIOR
# ====================================================================

# Guardar los embeddings como archivo .npy (formato binario eficiente de numpy)
np.save("outputs/brecha_embeddings.npy", brecha_embeddings)

# Guardar el DataFrame de brechas con índices para referencia posterior
# index=False: no incluye la columna de índice de pandas en el CSV
brechas.to_csv("outputs/brechas_with_idx.csv", index=False)

# Mostrar las dimensiones de la matriz de embeddings (número_brechas x dimensión_vector)
print("Embeddings shape:", brecha_embeddings.shape)
print(f"Modelo usado: {embed_model._model_card_vars.get('model_name', 'N/A')}")
print(f"Dimensiones por embedding: {brecha_embeddings.shape[1]}")


c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Batches: 100%|██████████| 2/2 [00:01<00:00,  1.34it/s]

Embeddings shape: (48, 768)
Modelo usado: N/A
Dimensiones por embedding: 768


In [8]:
# CELDA 4 - CREACIÓN DEL ÍNDICE FAISS PARA BÚSQUEDA VECTORIAL RÁPIDA
# ====================================================================
# FAISS (Facebook AI Similarity Search) es una librería optimizada para
# realizar búsquedas de vecinos más cercanos (k-NN) en espacios de alta dimensión
# ====================================================================

# ====================================================================
# CARGA DE EMBEDDINGS DESDE ARCHIVO
# ====================================================================

# Cargar los embeddings previamente generados en la celda 3
# np.load(): Lee el archivo .npy que contiene la matriz de embeddings
# astype("float32"): Convierte a float32 (FAISS requiere este tipo de dato para optimización)
emb = np.load("outputs/brecha_embeddings.npy").astype("float32")

# Obtener la dimensionalidad de los vectores (número de columnas)
# emb.shape[1] devuelve 384 (MiniLM) o 768 (MPNet), etc.
d = emb.shape[1]

# ====================================================================
# CREACIÓN DEL ÍNDICE FAISS
# ====================================================================

# IndexFlatIP: Índice FAISS que usa producto interno (Inner Product)
# - "Flat" significa búsqueda exacta (no aproximada), más preciso pero más lento
# - "IP" (Inner Product) en vectores normalizados es equivalente a similitud coseno
# - d: dimensión de los vectores (debe coincidir con la dimensión de los embeddings)
index = faiss.IndexFlatIP(d)

# Alternativas de índices FAISS:
# - faiss.IndexFlatL2(d): usa distancia L2 (euclidiana) en lugar de producto interno
# - faiss.IndexIVFFlat(quantizer, d, nlist): más rápido para muchos vectores (búsqueda aproximada)
# - faiss.IndexHNSWFlat(d, M): índice basado en grafos, muy rápido para alta dimensión

# ====================================================================
# AGREGAR VECTORES AL ÍNDICE
# ====================================================================

# Añadir todos los embeddings de brechas al índice FAISS
# Esto construye la estructura de datos interna para búsquedas eficientes
index.add(emb)

# ====================================================================
# GUARDAR EL ÍNDICE EN DISCO
# ====================================================================

# Guardar el índice FAISS como archivo binario .faiss
# Esto permite reutilizar el índice sin tener que reconstruirlo cada vez
faiss.write_index(index, "outputs/brecha_index.faiss")

# ====================================================================
# VERIFICACIÓN
# ====================================================================

# Mostrar el número total de vectores indexados
# index.ntotal debe ser igual al número de brechas (6 en tu caso)
print("Index ntotal:", index.ntotal)
print(f"Dimensión de vectores: {d}")
print(f"Tipo de índice: Inner Product (similitud coseno en vectores normalizados)")


Index ntotal: 48
Dimensión de vectores: 768
Tipo de índice: Inner Product (similitud coseno en vectores normalizados)


In [9]:
# CELDA 5 - PREPARACIÓN DE ETIQUETAS MULTIETIQUETA PARA CLASIFICACIÓN
# ====================================================================
# Objetivo: Convertir las cadenas de ids de brecha ("1,3,5") asociadas a cada
# proyecto en un formato matricial binario (one-hot multi-label) para entrenamiento.
# ====================================================================

# Estructura esperada de la columna proyectos["brecha_ids"]:
#   - Cada fila contiene una cadena con IDs separados por coma (ej: "2,4" o "1")
#   - Puede haber espacios o strings vacíos; se limpian en el proceso
# Resultado deseado (proyectos["brecha_ids_list"]): lista de ints por fila
#   Ej: "2,4" -> [2,4]; "1" -> [1]; "" -> []

# --------------------------------------------------------------------
# Paso 1: Normalizar y convertir las cadenas en listas de enteros
# --------------------------------------------------------------------
# .astype(str): asegura que todos los valores sean string (evita errores si hay NaN)
# .apply(lambda ...): procesa cada string:
#   - s.split(",") separa por comas -> ["2","4"]
#   - x.strip() elimina espacios
#   - if x.strip() != '' filtra vacíos (por ejemplo si había ",,")
#   - int(x) convierte cada fragmento a entero
proyectos["brecha_ids_list"] = proyectos["brecha_ids"].astype(str).apply(
    lambda s: [int(x) for x in s.split(",") if x.strip() != '']
)

# --------------------------------------------------------------------
# Paso 2: Inicializar MultiLabelBinarizer con el universo ordenado de clases
# --------------------------------------------------------------------
# MultiLabelBinarizer transforma listas de labels en una matriz binaria:
#   Ejemplo:
#       clases = [1,2,3,4]
#       entrada: [1,3] -> [1,0,1,0]
# sorted(brechas["id"].tolist()) asegura orden consistente de columnas.
mlb = MultiLabelBinarizer(classes=sorted(brechas["id"].tolist()))

# --------------------------------------------------------------------
# Paso 3: Fit + Transform
# --------------------------------------------------------------------
# mlb.fit_transform(lista_de_listas) genera la matriz Y:
#   - Filas: proyectos
#   - Columnas: cada brecha (en el orden de mlb.classes_)
#   - Valores: 1 si la brecha está asociada al proyecto, 0 si no.
Y = mlb.fit_transform(proyectos["brecha_ids_list"])

# --------------------------------------------------------------------
# Paso 4: Inspección rápida
# --------------------------------------------------------------------
# Forma esperada: (n_proyectos, n_brechas)
print("Shape labels:", Y.shape)

# Información adicional útil (descomentar si necesitas ver más detalles):
# print("Clases (orden):", mlb.classes_)
# print("Ejemplo primera fila lista original:", proyectos["brecha_ids_list"].iloc[0])
# print("Vector binario primera fila:", Y[0])


Shape labels: (817, 48)


In [10]:
# CELDA 6 - PREPARACIÓN DE DATASET PARA ENTRENAMIENTO DE BERT
# ====================================================================
# Objetivo: Crear un Dataset de PyTorch que tokeniza los textos de proyectos
# y los asocia con sus etiquetas multi-label para entrenar el modelo BETO (BERT español)
# ====================================================================

# ====================================================================
# CARGA DEL TOKENIZADOR
# ====================================================================

# Cargar el tokenizador del modelo BETO (BERT pre-entrenado en español)
# "dccuchile/bert-base-spanish-wwm-cased":
#   - Modelo BERT entrenado en español por la Universidad de Chile
#   - "wwm" = Whole Word Masking (enmascara palabras completas durante pre-entrenamiento)
#   - "cased" = distingue mayúsculas/minúsculas (importante para nombres propios)
tokenizer = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")

# ====================================================================
# DEFINICIÓN DE LA CLASE DATASET PERSONALIZADA
# ====================================================================

class BrechaDataset(torch.utils.data.Dataset):
    """
    Dataset personalizado para clasificación multi-label de proyectos en brechas.
    
    Hereda de torch.utils.data.Dataset para integrarse con DataLoader de PyTorch.
    Cada elemento del dataset contiene:
    - Texto tokenizado del proyecto (input_ids, attention_mask, etc.)
    - Vector binario de etiquetas (labels) indicando qué brechas aplican
    """
    
    def __init__(self, texts, labels, tokenizer, max_len=256):
        """
        Inicializa el dataset.
        
        Args:
            texts (list): Lista de strings con los textos de proyectos
            labels (numpy.ndarray): Matriz binaria (n_proyectos x n_brechas) de etiquetas
            tokenizer: Tokenizador de HuggingFace para convertir texto a tokens
            max_len (int): Longitud máxima de secuencia (tokens más allá se truncan)
        """
        self.texts = texts          # Textos originales de los proyectos
        self.labels = labels        # Matriz Y de etiquetas binarias
        self.tokenizer = tokenizer  # Tokenizador BETO
        self.max_len = max_len      # Longitud máxima permitida (256 tokens)

    def __len__(self):
        """Retorna el número total de ejemplos en el dataset."""
        return len(self.texts)

    def __getitem__(self, idx):
        """
        Obtiene un ejemplo del dataset en el índice dado.
        
        Args:
            idx (int): Índice del ejemplo a recuperar
            
        Returns:
            dict: Diccionario con:
                - 'input_ids': IDs de tokens del texto
                - 'attention_mask': Máscara de atención (1 para tokens reales, 0 para padding)
                - 'token_type_ids': IDs de tipo de token (para modelos BERT)
                - 'labels': Vector binario de etiquetas (float para BCEWithLogitsLoss)
        """
        # Convertir el texto a string (por si hay valores NaN o numéricos)
        text = str(self.texts[idx])
        
        # Tokenizar el texto usando el tokenizador BETO
        # - truncation=True: corta el texto si excede max_len
        # - padding='max_length': rellena con tokens especiales hasta max_len
        # - max_length=self.max_len: longitud objetivo (256)
        # - return_tensors='pt': retorna tensores de PyTorch
        inputs = self.tokenizer(
            text, 
            truncation=True, 
            padding='max_length', 
            max_length=self.max_len, 
            return_tensors='pt'
        )
        
        # Convertir tensores de forma (1, seq_len) a (seq_len,)
        # squeeze(0) elimina la dimensión batch añadida por return_tensors='pt'
        item = {k: v.squeeze(0) for k, v in inputs.items()}
        
        # Agregar las etiquetas como tensor float (requerido para pérdida BCEWithLogits)
        # labels[idx] es un array numpy de 0s y 1s indicando presencia de cada brecha
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        
        return item

# ====================================================================
# PREPARACIÓN DE TEXTOS DE ENTRADA
# ====================================================================

# Concatenar título y descripción de cada proyecto con un punto separador
# .fillna("") reemplaza valores NaN con string vacío para evitar errores
# Formato resultante: "TITULO DEL PROYECTO. Descripción detallada del proyecto..."
texts = (proyectos["title"].fillna("") + ". " + proyectos["description"].fillna("")).tolist()

# ====================================================================
# CREACIÓN DEL DATASET COMPLETO
# ====================================================================

# Instanciar el dataset con todos los proyectos y sus etiquetas
dataset = BrechaDataset(texts, Y, tokenizer)

# ====================================================================
# DIVISIÓN TRAIN/VALIDATION
# ====================================================================

# Importar función para dividir datos
from sklearn.model_selection import train_test_split

# Dividir índices en conjuntos de entrenamiento (85%) y validación (15%)
# - range(len(dataset)): genera índices [0, 1, 2, ..., n-1]
# - test_size=0.15: 15% de datos para validación
# - random_state=42: semilla aleatoria para reproducibilidad
idx_train, idx_val = train_test_split(
    range(len(dataset)), 
    test_size=0.15, 
    random_state=42
)

# ====================================================================
# CREACIÓN DE SUBSETS PARA ENTRENAMIENTO Y VALIDACIÓN
# ====================================================================

# Importar Subset para crear vistas del dataset original
from torch.utils.data import Subset

# Crear subset de entrenamiento con índices seleccionados
train_dataset = Subset(dataset, idx_train)

# Crear subset de validación con índices seleccionados
val_dataset = Subset(dataset, idx_val)

# Resumen de datasets creados
print(f"Dataset total: {len(dataset)} proyectos")
print(f"Train: {len(train_dataset)} proyectos ({len(train_dataset)/len(dataset)*100:.1f}%)")
print(f"Validación: {len(val_dataset)} proyectos ({len(val_dataset)/len(dataset)*100:.1f}%)")


Dataset total: 817 proyectos
Train: 694 proyectos (84.9%)
Validación: 123 proyectos (15.1%)


In [11]:
# CELDA 7 - ENTRENAMIENTO DEL MODELO BERT PARA CLASIFICACIÓN MULTI-LABEL
# ====================================================================
# Objetivo: Entrenar el modelo BETO (BERT español) para clasificar proyectos
# en múltiples brechas simultáneamente usando los datasets preparados en CELDA 6
# ====================================================================

# ====================================================================
# CARGA Y CONFIGURACIÓN DEL MODELO BASE
# ====================================================================

# Cargar modelo BERT pre-entrenado y configurarlo para clasificación multi-label
# AutoModelForSequenceClassification.from_pretrained():
#   - "dccuchile/bert-base-spanish-wwm-cased": modelo BETO pre-entrenado
#   - problem_type="multi_label_classification": indica que es multi-label (no multi-class)
#     Esto hace que internamente use BCEWithLogitsLoss en lugar de CrossEntropyLoss
#   - num_labels=Y.shape[1]: número de clases (6 brechas en tu caso)
#     Determina el tamaño de la capa de salida del clasificador
model = AutoModelForSequenceClassification.from_pretrained(
    "dccuchile/bert-base-spanish-wwm-cased",
    problem_type="multi_label_classification",  # Multi-label: cada proyecto puede tener múltiples brechas
    num_labels=Y.shape[1]  # Número de brechas distintas (columnas de la matriz Y)
)

# ====================================================================
# CONFIGURACIÓN DE HIPERPARÁMETROS DE ENTRENAMIENTO
# ====================================================================

# TrainingArguments: Clase que encapsula todos los hiperparámetros del entrenamiento
training_args = TrainingArguments(
    # Directorio donde se guardarán los checkpoints del modelo durante entrenamiento
    output_dir="models/beto_brechas",
    
    # Tamaño del batch por dispositivo (GPU/CPU) durante entrenamiento
    # 8 proyectos procesados simultáneamente (ajustar según VRAM disponible)
    per_device_train_batch_size=8,
    
    # Tamaño del batch durante evaluación (puede ser mayor porque no usa gradientes)
    # 16 proyectos procesados simultáneamente en validación
    per_device_eval_batch_size=16,
    
    # Estrategia de evaluación: "epoch" = evaluar al final de cada época
    # Alternativas: "steps" (cada N pasos), "no" (sin evaluación)
    evaluation_strategy="epoch",
    
    # Estrategia de guardado: "epoch" = guardar checkpoint al final de cada época
    # Creará subcarpetas checkpoint-1, checkpoint-2, checkpoint-3
    save_strategy="epoch",
    
    # Número de épocas de entrenamiento (pasadas completas por todo el dataset)
    # 3 épocas es típico para fine-tuning de BERT
    num_train_epochs=3,
    
    # Tasa de aprendizaje (learning rate) para el optimizador Adam
    # 2e-5 (0.00002) es el valor estándar recomendado para fine-tuning de BERT
    learning_rate=2e-5,
    
    # Directorio donde se guardarán los logs de TensorBoard
    # Útil para visualizar pérdida, métricas, etc. durante entrenamiento
    logging_dir="logs",
    
    # Cargar el mejor modelo al finalizar el entrenamiento (según metric_for_best_model)
    # True = al final, el modelo en memoria será el mejor checkpoint, no el último
    load_best_model_at_end=True,
    
    # Métrica para determinar cuál es el "mejor" modelo
    # "eval_loss" = menor pérdida en validación (alternativas: "eval_f1_micro", etc.)
    metric_for_best_model="eval_loss"
)

# ====================================================================
# FUNCIÓN DE CÁLCULO DE MÉTRICAS PARA EVALUACIÓN
# ====================================================================

def compute_metrics(eval_pred):
    """
    Calcula métricas de clasificación multi-label durante evaluación.
    
    Esta función es llamada automáticamente por Trainer al final de cada época
    para evaluar el rendimiento del modelo en el conjunto de validación.
    
    Args:
        eval_pred (tuple): Tupla (logits, labels) donde:
            - logits: salidas crudas del modelo (antes de sigmoid), shape (n_samples, n_labels)
            - labels: etiquetas verdaderas binarias, shape (n_samples, n_labels)
    
    Returns:
        dict: Diccionario con métricas calculadas:
            - f1_micro: F1-score micro-promediado (trata todas las predicciones como un conjunto)
            - precision_micro: Precisión micro-promediada
            - recall_micro: Recall micro-promediado
    """
    # Desempaquetar predicciones y etiquetas verdaderas
    logits, labels = eval_pred
    
    # Importar numpy localmente (ya está importado globalmente, pero por claridad)
    import numpy as np
    
    # Convertir logits a probabilidades usando función sigmoide
    # sigmoid(x) = 1 / (1 + e^(-x))
    # Transforma logits (-∞, +∞) a probabilidades [0, 1]
    probs = 1 / (1 + np.exp(-logits))
    
    # Convertir probabilidades a predicciones binarias usando umbral 0.5
    # prob >= 0.5 -> 1 (brecha asignada)
    # prob < 0.5 -> 0 (brecha no asignada)
    preds = (probs >= 0.5).astype(int)
    
    # Importar métricas de scikit-learn
    from sklearn.metrics import f1_score, precision_score, recall_score
    
    # Calcular y retornar métricas:
    # - average='micro': calcula métricas globalmente contando totales de TP, FP, FN
    #   (trata todo como una clasificación binaria grande)
    # - zero_division=0: si hay división por cero (no hay predicciones), retornar 0
    return {
        # F1-score: media armónica de precisión y recall (2*P*R / (P+R))
        "f1_micro": f1_score(labels, preds, average='micro', zero_division=0),
        
        # Precisión: TP / (TP + FP) - de las brechas predichas, cuántas son correctas
        "precision_micro": precision_score(labels, preds, average='micro', zero_division=0),
        
        # Recall: TP / (TP + FN) - de las brechas verdaderas, cuántas fueron detectadas
        "recall_micro": recall_score(labels, preds, average='micro', zero_division=0),
    }

# ====================================================================
# INICIALIZACIÓN DEL ENTRENADOR (TRAINER)
# ====================================================================

# Trainer: Clase de alto nivel de HuggingFace que encapsula el loop de entrenamiento
# Maneja automáticamente: forward pass, backward pass, optimización, evaluación, logging, checkpoints
trainer = Trainer(
    model=model,                    # Modelo BETO a entrenar
    args=training_args,             # Configuración de hiperparámetros
    train_dataset=train_dataset,    # Dataset de entrenamiento (85% de datos)
    eval_dataset=val_dataset,       # Dataset de validación (15% de datos)
    compute_metrics=compute_metrics # Función para calcular métricas en evaluación
)

# ====================================================================
# EJECUCIÓN DEL ENTRENAMIENTO
# ====================================================================

# Iniciar el proceso de entrenamiento
# Esto ejecutará:
#   1. num_train_epochs (3) épocas de entrenamiento
#   2. Evaluación al final de cada época (evaluation_strategy="epoch")
#   3. Guardado de checkpoints al final de cada época (save_strategy="epoch")
#   4. Cálculo de métricas (F1, precisión, recall) en cada evaluación
#   5. Selección del mejor modelo según eval_loss (load_best_model_at_end=True)
# 
# NOTA: Este proceso puede tardar varios minutos u horas según:
#   - Tamaño del dataset (número de proyectos)
#   - Hardware disponible (GPU vs CPU)
#   - Hiperparámetros (batch_size, num_epochs)
# 
# Progreso mostrado: barra de progreso con loss, learning rate, samples/sec
trainer.train()

# ====================================================================
# GUARDADO DEL MEJOR MODELO ENTRENADO
# ====================================================================

# Guardar el mejor modelo (según eval_loss) en disco
# Se guardarán:
#   - config.json: configuración del modelo
#   - pytorch_model.bin: pesos del modelo entrenado
#   - tokenizer_config.json, vocab.txt, etc.: archivos del tokenizador
# 
# Este modelo puede ser recargado más tarde con:
# AutoModelForSequenceClassification.from_pretrained("models/beto_brechas_best")
trainer.save_model("models/beto_brechas_best")

# Guardar también el tokenizador en el mismo directorio
# Esto permite cargar todo junto en la CELDA 8 sin errores
tokenizer.save_pretrained("models/beto_brechas_best")

# ====================================================================
# VERIFICACIÓN POST-ENTRENAMIENTO
# ====================================================================

print("\n" + "="*60)
print("✅ ENTRENAMIENTO COMPLETADO")
print("="*60)
print(f"Modelo guardado en: models/beto_brechas_best")
print(f"Checkpoints intermedios en: models/beto_brechas/checkpoint-*")
print(f"Logs de entrenamiento en: logs/")
print("="*60)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
                                                
 33%|███▎      | 87/261 [00:26<00:43,  3.99it/s]
                 

{'eval_loss': 0.19434063136577606, 'eval_f1_micro': 0.0, 'eval_precision_micro': 0.0, 'eval_recall_micro': 0.0, 'eval_runtime': 1.1849, 'eval_samples_per_second': 103.803, 'eval_steps_per_second': 6.751, 'epoch': 1.0}


                                                 
                                                 
 67%|██████▋   | 174/261 [00:52<00:19,  4.39it/s]

{'eval_loss': 0.13839370012283325, 'eval_f1_micro': 0.0, 'eval_precision_micro': 0.0, 'eval_recall_micro': 0.0, 'eval_runtime': 1.085, 'eval_samples_per_second': 113.364, 'eval_steps_per_second': 7.373, 'epoch': 2.0}


                                                 
                                                 
100%|██████████| 261/261 [01:18<00:00,  4.60it/s]

{'eval_loss': 0.1277291178703308, 'eval_f1_micro': 0.0, 'eval_precision_micro': 0.0, 'eval_recall_micro': 0.0, 'eval_runtime': 1.0182, 'eval_samples_per_second': 120.801, 'eval_steps_per_second': 7.857, 'epoch': 3.0}


100%|██████████| 261/261 [01:20<00:00,  3.26it/s]



{'train_runtime': 80.0197, 'train_samples_per_second': 26.019, 'train_steps_per_second': 3.262, 'train_loss': 0.20717234995173311, 'epoch': 3.0}

✅ ENTRENAMIENTO COMPLETADO
Modelo guardado en: models/beto_brechas_best
Checkpoints intermedios en: models/beto_brechas/checkpoint-*
Logs de entrenamiento en: logs/

✅ ENTRENAMIENTO COMPLETADO
Modelo guardado en: models/beto_brechas_best
Checkpoints intermedios en: models/beto_brechas/checkpoint-*
Logs de entrenamiento en: logs/


In [14]:
# CELDA 8 - CLASIFICACIÓN HÍBRIDA: COMBINACIÓN DE BÚSQUEDA SEMÁNTICA + BERT
# ====================================================================
# Objetivo: Implementar un sistema híbrido que combine la búsqueda vectorial (FAISS)
# con la clasificación BERT para obtener mejores predicciones que cada método por separado
# ====================================================================

# ====================================================================
# CARGA DE RECURSOS NECESARIOS PARA INFERENCIA
# ====================================================================

# Cargar índice FAISS con embeddings de brechas (creado en CELDA 4)
# Este índice permite búsqueda rápida de las brechas más similares semánticamente
index = faiss.read_index("outputs/brecha_index.faiss")

# Cargar DataFrame de brechas con sus IDs y textos
# Necesario para mapear los índices de FAISS a brechas legibles
brechas = pd.read_csv("outputs/brechas_with_idx.csv")

# Cargar modelo de embeddings para convertir el texto del proyecto en vector
# IMPORTANTE: Debe ser el MISMO modelo usado en CELDA 3 para mantener consistencia
# Si cambiaste a MPNet en CELDA 3, cámbialo aquí también
embed_model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-mpnet-base-v2")

# Cargar tokenizador del modelo BERT entrenado
# Convierte texto a tokens compatibles con el modelo BETO entrenado
bert_tokenizer = AutoTokenizer.from_pretrained("models/beto_brechas_best")

# Cargar modelo BERT entrenado (fine-tuned) para clasificación multi-label
# Este modelo fue guardado en CELDA 7 después del entrenamiento
bert_model = AutoModelForSequenceClassification.from_pretrained("models/beto_brechas_best")

# Poner el modelo en modo evaluación (desactiva dropout, batch norm, etc.)
# CRÍTICO: En modo eval(), el modelo no actualiza pesos y da resultados determinísticos
bert_model.eval()

# ====================================================================
# FUNCIÓN PRINCIPAL DE CLASIFICACIÓN HÍBRIDA
# ====================================================================

def hybrid_classify(project_text, top_k=5):
    """
    Clasifica un proyecto usando enfoque híbrido: búsqueda vectorial + BERT.
    
    Estrategia de dos etapas:
    1. RECUPERACIÓN (Retrieval): Usar embeddings + FAISS para encontrar top_k candidatos
    2. RE-RANKING: Usar BERT para puntuar candidatos y combinar scores
    
    Esta arquitectura es más eficiente que evaluar BERT en todas las brechas,
    y más precisa que usar solo búsqueda vectorial.
    
    Args:
        project_text (str): Texto del proyecto a clasificar (título + descripción)
        top_k (int): Número de candidatos a recuperar en la fase 1 (default: 5)
    
    Returns:
        list[dict]: Lista ordenada de brechas candidatas con scores, formato:
            [
                {
                    "brecha_id": int,           # ID de la brecha
                    "brecha_text": str,         # Texto descriptivo de la brecha
                    "bert_score": float,        # Probabilidad según BERT [0-1]
                    "embed_score": float,       # Similitud coseno según embeddings [0-1]
                    "final_score": float        # Puntaje combinado (alpha*BERT + beta*embed)
                },
                ...
            ]
            Ordenado de mayor a menor final_score
    """
    
    # ====================================================================
    # FASE 1: RECUPERACIÓN DE CANDIDATOS CON BÚSQUEDA VECTORIAL
    # ====================================================================
    
    # Convertir el texto del proyecto a vector usando el mismo modelo de embeddings
    # - [project_text]: lista de un elemento (el modelo espera lista)
    # - normalize_embeddings=True: normalizar a norma L2=1 (igual que en CELDA 3)
    # - astype("float32"): FAISS requiere float32 para búsqueda eficiente
    qv = embed_model.encode([project_text], normalize_embeddings=True).astype("float32")
    
    # Buscar las top_k brechas más similares en el índice FAISS
    # index.search(query_vectors, k) retorna:
    #   - scores: distancias/similitudes (producto interno en este caso)
    #   - idxs: índices de los vectores más cercanos en el índice
    # Forma: scores y idxs son arrays de shape (1, top_k)
    scores, idxs = index.search(qv, top_k)
    
    # Extraer los índices como lista Python (idxs[0] porque query tiene batch_size=1)
    # candidate_idxs: [3, 1, 5, 2, 4] por ejemplo (índices en el DataFrame brechas)
    candidate_idxs = idxs[0].tolist()

    # ====================================================================
    # FASE 2: SCORING CON BERT SOBRE TODAS LAS BRECHAS
    # ====================================================================
    
    # Tokenizar el texto del proyecto para BERT
    # - return_tensors="pt": retornar tensores de PyTorch
    # - truncation=True: cortar si excede max_length
    # - padding=True: rellenar hasta max_length
    # - max_length=256: misma longitud usada en entrenamiento
    inputs = bert_tokenizer(project_text, return_tensors="pt", truncation=True, padding=True, max_length=256)
    
    # Ejecutar modelo BERT en modo inferencia (sin calcular gradientes)
    # torch.no_grad(): desactiva cálculo de gradientes (ahorra memoria y acelera)
    with torch.no_grad():
        # Forward pass del modelo BERT
        # .logits: salidas crudas antes de sigmoid, shape (1, n_labels)
        # .cpu().numpy(): mover tensor a CPU y convertir a numpy array
        # [0]: extraer el primer elemento (batch size = 1)
        logits = bert_model(**inputs).logits.cpu().numpy()[0]
        
        # Convertir logits a probabilidades usando función sigmoide
        # sigmoid(x) = 1 / (1 + e^(-x))
        # probs[i] = probabilidad de que la brecha i aplique al proyecto
        # Valores entre [0, 1] donde > 0.5 típicamente significa "brecha asignada"
        probs = 1 / (1 + np.exp(-logits))

    # ====================================================================
    # FASE 3: COMBINACIÓN DE SCORES Y RE-RANKING
    # ====================================================================
    
    # Lista para almacenar resultados con scores combinados
    results = []
    
    # Iterar sobre cada candidato recuperado en la Fase 1
    for rank, cand in enumerate(candidate_idxs):
        # Obtener fila completa de la brecha desde el DataFrame
        # cand es el índice (0-based) en el DataFrame brechas
        brecha_row = brechas.iloc[cand]
        
        # Calcular índice en el espacio de labels del modelo BERT
        # Si los IDs de brechas empiezan en 1, restar 1 para obtener índice 0-based
        # Ejemplo: brecha_id=1 -> label_index=0, brecha_id=6 -> label_index=5
        label_index = int(brecha_row["id"]) - 1
        
        # Extraer el score BERT para esta brecha específica
        # probs[label_index]: probabilidad que BERT asigna a esta brecha
        bert_score = float(probs[label_index])
        
        # Extraer el score de similitud semántica (embeddings)
        # scores[0][rank]: similitud coseno (producto interno de vectores normalizados)
        # Valores cercanos a 1 = muy similar, cercanos a 0 = poco similar
        embed_score = float(scores[0][rank])
        
        # Combinar ambos scores usando promedio ponderado
        # alpha: peso para BERT (énfasis en clasificación supervisada)
        # beta: peso para embeddings (énfasis en similitud semántica)
        # alpha + beta debe sumar 1.0 idealmente
        alpha, beta = 0.6, 0.4  # 60% BERT, 40% embeddings
        
        # AJUSTE DE HIPERPARÁMETROS alpha/beta:
        # - Aumentar alpha si BERT tiene buen rendimiento en validación
        # - Aumentar beta si la similitud semántica es más confiable
        # - Probar valores como (0.7, 0.3), (0.5, 0.5), (0.8, 0.2)
        final_score = alpha * bert_score + beta * embed_score
        
        # Agregar resultado con toda la información
        results.append({
            "brecha_id": brecha_row["id"],          # ID numérico de la brecha
            "brecha_text": brecha_row["brecha"],    # Descripción textual
            "bert_score": bert_score,                # Score del modelo supervisado
            "embed_score": embed_score,              # Score de similitud semántica
            "final_score": final_score               # Score combinado final
        })
    
    # ====================================================================
    # ORDENAMIENTO FINAL POR SCORE COMBINADO
    # ====================================================================
    
    # Ordenar resultados de mayor a menor final_score
    # La brecha con mayor final_score es la predicción principal
    results = sorted(results, key=lambda x: x["final_score"], reverse=True)
    
    return results

# ====================================================================
# EJEMPLO DE USO Y PRUEBA CON PROYECTOS REALES DEL DATASET
# ====================================================================

# Cargar proyectos para testing
proyectos_test = pd.read_csv("data/proyectos.csv")

# Seleccionar un proyecto real del dataset para validar el modelo
# Vamos a probar con el proyecto ID 211 que es de educación inicial (brecha 22)
test_idx = 210  # Índice 210 = fila 211 en el CSV (0-based)
proyecto_real = proyectos_test.iloc[test_idx]

# Concatenar título y descripción como lo hacemos en entrenamiento
text_proyecto = f"{proyecto_real['title']}. {proyecto_real['description']}"

# Brecha verdadera (ground truth)
brecha_verdadera = proyecto_real['brecha_ids']

print("\n" + "="*80)
print("PRUEBA DE CLASIFICACIÓN HÍBRIDA - PROYECTO REAL DEL DATASET")
print("="*80)
print(f"Project ID: {proyecto_real['project_id']}")
print(f"Título: {proyecto_real['title']}")
print(f"\nBRECHA VERDADERA: {brecha_verdadera}")
print(f"Descripción brecha: {brechas[brechas['id'] == int(brecha_verdadera)]['brecha'].values[0]}")
print("\n" + "-"*80)
print("Resultados de clasificación (top 5 candidatos):")
print("-"*80)

# Ejecutar clasificación híbrida
clasificacion = hybrid_classify(text_proyecto, top_k=5)

# Mostrar cada candidato con sus scores
for i, resultado in enumerate(clasificacion, 1):
    # Marcar con ✅ si es la brecha correcta
    marcador = "✅ CORRECTO" if resultado['brecha_id'] == int(brecha_verdadera) else ""
    print(f"\n{i}. Brecha ID {resultado['brecha_id']}: {resultado['brecha_text'][:80]}... {marcador}")
    print(f"   - BERT score:       {resultado['bert_score']:.4f}")
    print(f"   - Embedding score:  {resultado['embed_score']:.4f}")
    print(f"   - FINAL score:      {resultado['final_score']:.4f}")

print("\n" + "="*80)
prediccion_correcta = clasificacion[0]['brecha_id'] == int(brecha_verdadera)
if prediccion_correcta:
    print(f"✅ PREDICCIÓN CORRECTA: Brecha ID {clasificacion[0]['brecha_id']}")
else:
    print(f"❌ PREDICCIÓN INCORRECTA")
    print(f"   Predijo: Brecha ID {clasificacion[0]['brecha_id']}")
    print(f"   Esperado: Brecha ID {brecha_verdadera}")
print("="*80)

# ====================================================================
# PRUEBA ADICIONAL: Proyecto de ejemplo que mencionaste
# ====================================================================

print("\n\n" + "="*80)
print("PRUEBA CON PROYECTO DE EJEMPLO (CUNA JARDIN)")
print("="*80)

proj = "AMPLIACION DE 01 AULA + 01 DIRECCION EN LA CUNA JARDIN, HOSPITAL AMAZONICO- PUERTO CALLAO"
print(f"Proyecto: {proj}\n")

# Nota: Este proyecto debería ser brecha 48 (educación inicial - matriculación)
# pero esa brecha NO EXISTE en el dataset de entrenamiento, por lo que el modelo
# probablemente clasificará como brecha 22 (infraestructura educación inicial)
# o brecha 5 (centros cuna más)

print("⚠️ NOTA: Este proyecto menciona 'CUNA JARDIN' (educación inicial)")
print("Las brechas relacionadas en el dataset son:")
print("  - Brecha 5: CENTROS CUNA MÁS EN CONDICIONES INADECUADAS")
print("  - Brecha 22: LOCALES EDUCATIVOS CON SERVICIO DE EDUCACIÓN INICIAL INADECUADA")
print("  - Brecha 48: PERSONAS NO MATRICULADAS EN NIVEL INICIAL (⚠️ SIN DATOS DE ENTRENAMIENTO)")
print("\n" + "-"*80)
print("Resultados (top 5 candidatos):")
print("-"*80)

# Llamar función y mostrar resultados
clasificacion2 = hybrid_classify(proj, top_k=5)

# Mostrar cada candidato con sus scores
for i, resultado in enumerate(clasificacion2, 1):
    print(f"\n{i}. Brecha ID {resultado['brecha_id']}: {resultado['brecha_text'][:80]}...")
    print(f"   - BERT score:       {resultado['bert_score']:.4f}")
    print(f"   - Embedding score:  {resultado['embed_score']:.4f}")
    print(f"   - FINAL score:      {resultado['final_score']:.4f}")

print("\n" + "="*80)
print(f"✅ Predicción principal: Brecha ID {clasificacion2[0]['brecha_id']}")
print("="*80)

# ====================================================================
# ANÁLISIS DE DISTRIBUCIÓN DE BRECHAS EN EL DATASET
# ====================================================================

print("\n\n" + "="*80)
print("ANÁLISIS DE DISTRIBUCIÓN DE BRECHAS EN EL DATASET DE ENTRENAMIENTO")
print("="*80)

# Contar proyectos por brecha
brecha_counts = {}
for idx, row in proyectos_test.iterrows():
    brecha_ids = str(row['brecha_ids']).split(',')
    for bid in brecha_ids:
        bid = bid.strip()
        if bid:
            bid_int = int(bid)
            brecha_counts[bid_int] = brecha_counts.get(bid_int, 0) + 1

# Ordenar por frecuencia
brecha_counts_sorted = sorted(brecha_counts.items(), key=lambda x: x[1], reverse=True)

print(f"\nTotal de proyectos: {len(proyectos_test)}")
print(f"Total de brechas en catálogo: {len(brechas)}")
print(f"Brechas con al menos 1 proyecto: {len(brecha_counts)}")
print(f"Brechas sin proyectos: {len(brechas) - len(brecha_counts)}")

print("\n📊 Top 10 brechas más frecuentes:")
for i, (bid, count) in enumerate(brecha_counts_sorted[:10], 1):
    brecha_text = brechas[brechas['id'] == bid]['brecha'].values[0]
    print(f"{i}. Brecha {bid}: {count} proyectos - {brecha_text[:60]}...")

print("\n⚠️ Brechas sin ejemplos de entrenamiento:")
brechas_sin_datos = [bid for bid in brechas['id'] if bid not in brecha_counts]
if brechas_sin_datos:
    for bid in brechas_sin_datos[:5]:  # Mostrar solo las primeras 5
        brecha_text = brechas[brechas['id'] == bid]['brecha'].values[0]
        print(f"  - Brecha {bid}: {brecha_text[:70]}...")
    if len(brechas_sin_datos) > 5:
        print(f"  ... y {len(brechas_sin_datos) - 5} brechas más sin datos")
else:
    print("  ✅ Todas las brechas tienen al menos un proyecto")

print("="*80)


c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(



PRUEBA DE CLASIFICACIÓN HÍBRIDA - PROYECTO REAL DEL DATASET
Project ID: 211
Título: MEJORAMIENTO DEL SERVICIO DE EDUCACION INICIAL EN 03 I.E.I., DISTRITO DE SAN PEDRO DE HUACARPANA - CHINCHA - ICA

BRECHA VERDADERA: 22
Descripción brecha: PORCENTAJE DE LOCALES EDUCATIVOS CON EL SERVICIO DE EDUCACIÓN INICIAL CON CAPACIDAD INSTALADA INADECUADA

--------------------------------------------------------------------------------
Resultados de clasificación (top 5 candidatos):
--------------------------------------------------------------------------------

1. Brecha ID 23: PORCENTAJE DE LOCALES EDUCATIVOS CON EL SERVICIO DE EDUCACIÓN PRIMARIA CON CAPAC... 
   - BERT score:       0.1189
   - Embedding score:  0.7596
   - FINAL score:      0.3752

2. Brecha ID 7: PORCENTAJE DE CENTROS DE EDUCACIÓN BÁSICA ESPECIAL CON CAPACIDAD INSTALADA INADE... 
   - BERT score:       0.0802
   - Embedding score:  0.7679
   - FINAL score:      0.3553

3. Brecha ID 22: PORCENTAJE DE LOCALES EDUCATIVOS CON EL S

## 🔧 Mejora de Calibración de Scores

El problema que observaste es real: los scores están muy bajos. Esto se debe a:

1. **BERT scores sin normalizar**: Los logits crudos de BERT no son probabilidades
2. **Combinación simple**: Promediar scores de diferentes escalas no es óptimo
3. **Similitud coseno limitada**: Rara vez supera 0.8-0.9 en embeddings genéricos

### Soluciones que implementaremos:
- ✅ Aplicar **softmax** a los scores de BERT para obtener probabilidades reales
- ✅ Permitir **ajustar pesos** entre BERT y embeddings (no solo 50-50)
- ✅ **Normalizar** el score final a rango 0-1
- ✅ Mostrar **scores calibrados** que tengan sentido intuitivo

In [18]:
import torch.nn.functional as F

# Cargar los brecha_ids para el mapeo correcto
brecha_ids = brechas['id'].tolist()

def clasificar_hibrido_mejorado(
    texto_proyecto, 
    bert_model, 
    bert_tokenizer, 
    embed_model, 
    brecha_embeddings, 
    brecha_ids, 
    index,
    top_k=5,
    bert_weight=0.5,  # Peso de BERT (0-1), el resto es para embeddings
    use_softmax=True,  # Si True, aplica softmax a scores de BERT
    normalize_final=True  # Si True, normaliza el score final a 0-1
):
    """
    Versión mejorada con scores calibrados.
    
    Args:
        bert_weight: Peso para BERT (0-1). Si es 0.7, BERT tiene 70% y embeddings 30%
        use_softmax: Si True, convierte logits de BERT a probabilidades con softmax
        normalize_final: Si True, escala el score final al rango 0-1
    """
    # 1. BERT: Clasificación directa
    inputs = bert_tokenizer(
        texto_proyecto, 
        return_tensors='pt', 
        truncation=True, 
        max_length=512,
        padding=True
    )
    
    if torch.cuda.is_available():
        inputs = {k: v.cuda() for k, v in inputs.items()}
        bert_model = bert_model.cuda()
    
    with torch.no_grad():
        outputs = bert_model(**inputs)
        logits = outputs.logits[0]  # Shape: [num_classes]
        
        if use_softmax:
            # Aplicar softmax para obtener probabilidades reales
            bert_probs = F.softmax(logits, dim=0).cpu().numpy()
        else:
            # Usar logits crudos (método anterior)
            bert_probs = logits.cpu().numpy()
    
    # 2. Embeddings: Búsqueda semántica
    emb_proyecto = embed_model.encode([texto_proyecto], convert_to_numpy=True)
    emb_proyecto = emb_proyecto / np.linalg.norm(emb_proyecto, axis=1, keepdims=True)
    
    # Buscar vecinos más cercanos
    D, I = index.search(emb_proyecto.astype('float32'), top_k)
    
    # 3. Combinación híbrida
    resultados = []
    for rank_idx in range(min(top_k, len(I[0]))):
        brecha_idx = I[0][rank_idx]
        brecha_id = brecha_ids[brecha_idx]
        
        # Score de similitud semántica (ya es 0-1 por coseno normalizado)
        sim_score = float(D[0][rank_idx])
        
        # Score de BERT para esta brecha
        bert_score = float(bert_probs[brecha_id - 1])  # brecha_id empieza en 1
        
        # Combinar scores con pesos
        embedding_weight = 1.0 - bert_weight
        combined_score = (bert_weight * bert_score) + (embedding_weight * sim_score)
        
        resultados.append({
            'brecha_id': brecha_id,
            'bert_score': bert_score,
            'embedding_score': sim_score,
            'combined_score': combined_score,
            'bert_weight': bert_weight,
            'embedding_weight': embedding_weight
        })
    
    # Ordenar por score combinado
    resultados = sorted(resultados, key=lambda x: x['combined_score'], reverse=True)
    
    # 4. Normalización final (opcional)
    if normalize_final and len(resultados) > 0:
        max_score = max(r['combined_score'] for r in resultados)
        min_score = min(r['combined_score'] for r in resultados)
        score_range = max_score - min_score
        
        if score_range > 0:
            for r in resultados:
                r['combined_score_normalized'] = (r['combined_score'] - min_score) / score_range
        else:
            for r in resultados:
                r['combined_score_normalized'] = 1.0
    
    return resultados


print("✅ Función de clasificación mejorada definida")
print(f"✅ Brecha IDs cargados: {len(brecha_ids)} brechas")
print("\nParámetros configurables:")
print("  - bert_weight: Peso de BERT vs embeddings (default 0.5 = 50-50)")
print("  - use_softmax: Convertir logits a probabilidades (default True)")
print("  - normalize_final: Escalar score final a 0-1 (default True)")

✅ Función de clasificación mejorada definida
✅ Brecha IDs cargados: 48 brechas

Parámetros configurables:
  - bert_weight: Peso de BERT vs embeddings (default 0.5 = 50-50)
  - use_softmax: Convertir logits a probabilidades (default True)
  - normalize_final: Escalar score final a 0-1 (default True)


In [19]:
# ============================================================
# PRUEBA CON EL PROYECTO REAL - VERSIÓN MEJORADA
# ============================================================

test_idx = 211
proyecto_real = proyectos.iloc[test_idx]
brecha_verdadera = int(proyecto_real['brecha_ids'])  # Convertir a int por si acaso
text_proyecto = f"{proyecto_real['title']} {proyecto_real['description']}"

print("=" * 80)
print("PRUEBA CON SCORES CALIBRADOS - PROYECTO REAL")
print("=" * 80)
print(f"Project ID: {proyecto_real['project_id']}")
print(f"Título: {proyecto_real['title'][:100]}...")
print(f"\nBRECHA VERDADERA: {brecha_verdadera}")

brecha_desc = brechas[brechas['id'] == brecha_verdadera]['brecha'].values
if len(brecha_desc) > 0:
    print(f"Descripción brecha: {brecha_desc[0]}")

print("\n" + "-" * 80)
print("🔧 CONFIGURACIÓN 1: Softmax + Pesos balanceados (50-50)")
print("-" * 80)

resultados = clasificar_hibrido_mejorado(
    text_proyecto,
    bert_model,
    bert_tokenizer,
    embed_model,
    brecha_embeddings,
    brecha_ids,
    index,
    top_k=5,
    bert_weight=0.5,
    use_softmax=True,
    normalize_final=True
)

for i, r in enumerate(resultados, 1):
    bid = r['brecha_id']
    desc = brechas[brechas['id'] == bid]['brecha'].values[0][:80]
    
    marcador = "✅ CORRECTO" if bid == brecha_verdadera else ""
    
    print(f"\n{i}. Brecha ID {bid}: {desc}... {marcador}")
    print(f"   - BERT score:       {r['bert_score']:.4f}")
    print(f"   - Embedding score:  {r['embedding_score']:.4f}")
    print(f"   - Combined score:   {r['combined_score']:.4f}")
    if 'combined_score_normalized' in r:
        print(f"   - Normalized:       {r['combined_score_normalized']:.4f}")

prediccion = resultados[0]['brecha_id']
if prediccion == brecha_verdadera:
    print(f"\n✅ PREDICCIÓN CORRECTA: Brecha ID {prediccion}")
else:
    print(f"\n❌ PREDICCIÓN INCORRECTA")
    print(f"   Predijo: Brecha ID {prediccion}")
    print(f"   Esperado: Brecha ID {brecha_verdadera}")

print("\n" + "=" * 80)
print("🔧 CONFIGURACIÓN 2: Softmax + Mayor peso a BERT (70-30)")
print("=" * 80)

resultados2 = clasificar_hibrido_mejorado(
    text_proyecto,
    bert_model,
    bert_tokenizer,
    embed_model,
    brecha_embeddings,
    brecha_ids,
    index,
    top_k=5,
    bert_weight=0.7,  # Más peso a BERT
    use_softmax=True,
    normalize_final=True
)

for i, r in enumerate(resultados2, 1):
    bid = r['brecha_id']
    desc = brechas[brechas['id'] == bid]['brecha'].values[0][:80]
    
    marcador = "✅ CORRECTO" if bid == brecha_verdadera else ""
    
    print(f"\n{i}. Brecha ID {bid}: {desc}... {marcador}")
    print(f"   - BERT score:       {r['bert_score']:.4f}")
    print(f"   - Embedding score:  {r['embedding_score']:.4f}")
    print(f"   - Combined score:   {r['combined_score']:.4f}")
    if 'combined_score_normalized' in r:
        print(f"   - Normalized:       {r['combined_score_normalized']:.4f}")

prediccion2 = resultados2[0]['brecha_id']
if prediccion2 == brecha_verdadera:
    print(f"\n✅ PREDICCIÓN CORRECTA: Brecha ID {prediccion2}")
else:
    print(f"\n❌ PREDICCIÓN INCORRECTA")
    print(f"   Predijo: Brecha ID {prediccion2}")
    print(f"   Esperado: Brecha ID {brecha_verdadera}")

print("\n" + "=" * 80)
print("🔧 CONFIGURACIÓN 3: Softmax + Mayor peso a Embeddings (30-70)")
print("=" * 80)

resultados3 = clasificar_hibrido_mejorado(
    text_proyecto,
    bert_model,
    bert_tokenizer,
    embed_model,
    brecha_embeddings,
    brecha_ids,
    index,
    top_k=5,
    bert_weight=0.3,  # Más peso a embeddings
    use_softmax=True,
    normalize_final=True
)

for i, r in enumerate(resultados3, 1):
    bid = r['brecha_id']
    desc = brechas[brechas['id'] == bid]['brecha'].values[0][:80]
    
    marcador = "✅ CORRECTO" if bid == brecha_verdadera else ""
    
    print(f"\n{i}. Brecha ID {bid}: {desc}... {marcador}")
    print(f"   - BERT score:       {r['bert_score']:.4f}")
    print(f"   - Embedding score:  {r['embedding_score']:.4f}")
    print(f"   - Combined score:   {r['combined_score']:.4f}")
    if 'combined_score_normalized' in r:
        print(f"   - Normalized:       {r['combined_score_normalized']:.4f}")

prediccion3 = resultados3[0]['brecha_id']
if prediccion3 == brecha_verdadera:
    print(f"\n✅ PREDICCIÓN CORRECTA: Brecha ID {prediccion3}")
else:
    print(f"\n❌ PREDICCIÓN INCORRECTA")
    print(f"   Predijo: Brecha ID {prediccion3}")
    print(f"   Esperado: Brecha ID {brecha_verdadera}")

print("\n" + "=" * 80)

PRUEBA CON SCORES CALIBRADOS - PROYECTO REAL
Project ID: 212
Título: MEJORAMIENTO DEL SISTEMA DE AGUA POTABLE Y ALCANTARILLADO EN LA COMUNIDAD CAMPESINA DE VILAVILANI, D...

BRECHA VERDADERA: 17
Descripción brecha: PORCENTAJE DE LA POBLACIÓN RURAL SIN ACCESO AL SERVICIO DE ALCANTARILLADO U OTRAS FORMAS DE DISPOSICIÓN SANITARIA DE EXCRETAS

--------------------------------------------------------------------------------
🔧 CONFIGURACIÓN 1: Softmax + Pesos balanceados (50-50)
--------------------------------------------------------------------------------

1. Brecha ID 12: PORCENTAJE DE HORAS AL DÍA SIN SERVICIO DE AGUA POTABLE EN EL ÁMBITO URBANO... 
   - BERT score:       0.0184
   - Embedding score:  0.7182
   - Combined score:   0.3683
   - Normalized:       1.0000

2. Brecha ID 42: PORCENTAJE DE SUPERFICIE SIN ACONDICIONAMIENTO PARA RECARGA HÍDRICA PROVENIENTES... 
   - BERT score:       0.0121
   - Embedding score:  0.6511
   - Combined score:   0.3316
   - Normalized:       0.1705


## 📈 Interpretación de los Scores

### ¿Por qué los scores no llegan a 0.9?

#### BERT Scores (0.01 - 0.02 = 1-2%)
- **Son probabilidades reales** después de aplicar softmax
- **Bajos porque hay 48 clases**: Si fueran uniformes, cada una tendría 1/48 = 2.08%
- **El modelo no está seguro**: Indica que necesita más datos de entrenamiento
- **Normal con pocos datos**: 817 proyectos ÷ 48 clases = ~17 ejemplos/clase promedio

#### Embedding Scores (0.62 - 0.72 = 62-72%)
- **Son similitudes coseno** (0 = no similar, 1 = idéntico)
- **0.70+ es considerado "bueno"** para embeddings genéricos
- **No llegan a 0.9** porque el modelo no fue fine-tuned en tu dominio

#### ¿Qué significa esto?
- ✅ **Los scores están calibrados correctamente** ahora
- ⚠️ **BERT necesita más entrenamiento o datos** (scores muy bajos)
- ✅ **Embeddings funcionan bien** (0.7+ de similitud es respetable)
- 💡 **El sistema híbrido está balanceando ambos métodos**

### Opciones para mejorar:

1. **Más datos de entrenamiento** para BERT (conseguir más proyectos etiquetados)
2. **Fine-tune del modelo de embeddings** en tu dominio específico
3. **Ajustar pesos dinámicamente** según confianza del modelo
4. **Usar solo embeddings** si BERT no mejora (bert_weight=0.0)

In [20]:
# ============================================================
# COMPARACIÓN: SOLO EMBEDDINGS vs HÍBRIDO
# ============================================================
print("=" * 80)
print("🔍 COMPARACIÓN DE ESTRATEGIAS")
print("=" * 80)

# Estrategia 1: Solo embeddings (bert_weight=0.0)
print("\n1️⃣ SOLO EMBEDDINGS (bert_weight=0.0)")
print("-" * 80)

resultados_solo_emb = clasificar_hibrido_mejorado(
    text_proyecto,
    bert_model,
    bert_tokenizer,
    embed_model,
    brecha_embeddings,
    brecha_ids,
    index,
    top_k=5,
    bert_weight=0.0,  # 100% embeddings, 0% BERT
    use_softmax=True,
    normalize_final=False  # No normalizar para ver scores reales
)

for i, r in enumerate(resultados_solo_emb, 1):
    bid = r['brecha_id']
    desc = brechas[brechas['id'] == bid]['brecha'].values[0][:80]
    marcador = "✅ CORRECTO" if bid == brecha_verdadera else ""
    print(f"{i}. Brecha {bid}: {desc}... {marcador}")
    print(f"   Embedding score: {r['embedding_score']:.4f}")

prediccion_emb = resultados_solo_emb[0]['brecha_id']
print(f"\n{'✅' if prediccion_emb == brecha_verdadera else '❌'} Predicción: Brecha {prediccion_emb}")

# Estrategia 2: Solo BERT (bert_weight=1.0)
print("\n\n2️⃣ SOLO BERT (bert_weight=1.0)")
print("-" * 80)

resultados_solo_bert = clasificar_hibrido_mejorado(
    text_proyecto,
    bert_model,
    bert_tokenizer,
    embed_model,
    brecha_embeddings,
    brecha_ids,
    index,
    top_k=5,
    bert_weight=1.0,  # 100% BERT, 0% embeddings
    use_softmax=True,
    normalize_final=False
)

for i, r in enumerate(resultados_solo_bert, 1):
    bid = r['brecha_id']
    desc = brechas[brechas['id'] == bid]['brecha'].values[0][:80]
    marcador = "✅ CORRECTO" if bid == brecha_verdadera else ""
    print(f"{i}. Brecha {bid}: {desc}... {marcador}")
    print(f"   BERT score: {r['bert_score']:.4f}")

prediccion_bert = resultados_solo_bert[0]['brecha_id']
print(f"\n{'✅' if prediccion_bert == brecha_verdadera else '❌'} Predicción: Brecha {prediccion_bert}")

# Resumen
print("\n" + "=" * 80)
print("📊 RESUMEN DE PREDICCIONES")
print("=" * 80)
print(f"Brecha verdadera: {brecha_verdadera}")
print(f"  - Solo Embeddings: {prediccion_emb} {'✅' if prediccion_emb == brecha_verdadera else '❌'}")
print(f"  - Solo BERT:       {prediccion_bert} {'✅' if prediccion_bert == brecha_verdadera else '❌'}")
print(f"  - Híbrido 50-50:   {resultados[0]['brecha_id']} {'✅' if resultados[0]['brecha_id'] == brecha_verdadera else '❌'}")
print("=" * 80)

🔍 COMPARACIÓN DE ESTRATEGIAS

1️⃣ SOLO EMBEDDINGS (bert_weight=0.0)
--------------------------------------------------------------------------------
1. Brecha 12: PORCENTAJE DE HORAS AL DÍA SIN SERVICIO DE AGUA POTABLE EN EL ÁMBITO URBANO... 
   Embedding score: 0.7182
2. Brecha 42: PORCENTAJE DE SUPERFICIE SIN ACONDICIONAMIENTO PARA RECARGA HÍDRICA PROVENIENTES... 
   Embedding score: 0.6511
3. Brecha 4: PORCENTAJE DE CEMENTERIOS CON CAPACIDAD INSTALADA INADECUADA... 
   Embedding score: 0.6355
4. Brecha 31: PORCENTAJE DE M2 DE ESPACIOS PÚBLICOS VERDES POR HABITANTE EN LAS ZONAS URBANAS ... 
   Embedding score: 0.6343
5. Brecha 13: PORCENTAJE DE INSTALACIONES DEPORTIVAS Y /O RECREATIVAS EN CONDICIONES INADECUAD... 
   Embedding score: 0.6263

❌ Predicción: Brecha 12


2️⃣ SOLO BERT (bert_weight=1.0)
--------------------------------------------------------------------------------
1. Brecha 13: PORCENTAJE DE INSTALACIONES DEPORTIVAS Y /O RECREATIVAS EN CONDICIONES INADECUAD... 
   BERT 

## 📊 ¿Cuántos Registros Necesitas? ¿BETO es la Solución Correcta?

### Situación Actual
Tienes **817 proyectos** etiquetados para **48 brechas**

### 🎯 Regla General: Registros Recomendados por Clase

| Escenario | Registros/Clase | Total (48 clases) | Calidad Esperada |
|-----------|----------------|-------------------|------------------|
| **Mínimo viable** | 20-50 | 960-2,400 | Baja (~40-60% accuracy) ✅ TU CASO |
| **Aceptable** | 100-200 | 4,800-9,600 | Media (~65-75% accuracy) |
| **Bueno** | 500-1,000 | 24,000-48,000 | Alta (~80-85% accuracy) |
| **Excelente** | 2,000+ | 96,000+ | Muy alta (~90%+ accuracy) |

Tu promedio: **817 ÷ 48 = 17 ejemplos/clase** ⚠️ **BAJO EL MÍNIMO**

### 📈 Análisis de Tu Dataset

In [21]:
# ============================================================
# ANÁLISIS: ¿Cuántos datos tienes vs. cuántos necesitas?
# ============================================================

import pandas as pd
import numpy as np

# Contar distribución de brechas
brecha_distribution = proyectos['brecha_ids'].value_counts().sort_index()

print("=" * 80)
print("📊 ANÁLISIS DE TU DATASET")
print("=" * 80)
print(f"\nTotal de proyectos: {len(proyectos)}")
print(f"Total de brechas: {len(brechas)}")
print(f"Promedio por brecha: {len(proyectos) / len(brechas):.1f} proyectos")

# Estadísticas de distribución
print(f"\n📈 Distribución de ejemplos:")
print(f"  - Mínimo:  {brecha_distribution.min()} proyectos (brecha {brecha_distribution.idxmin()})")
print(f"  - Máximo:  {brecha_distribution.max()} proyectos (brecha {brecha_distribution.idxmax()})")
print(f"  - Mediana: {brecha_distribution.median():.0f} proyectos")
print(f"  - Media:   {brecha_distribution.mean():.1f} proyectos")

# Categorizar brechas por cantidad de datos
pocos_datos = brecha_distribution[brecha_distribution < 10]
datos_minimos = brecha_distribution[(brecha_distribution >= 10) & (brecha_distribution < 50)]
datos_aceptables = brecha_distribution[(brecha_distribution >= 50) & (brecha_distribution < 100)]
datos_buenos = brecha_distribution[brecha_distribution >= 100]

print(f"\n🔍 Categorización de brechas por cantidad de datos:")
print(f"  - Muy pocos datos (<10):      {len(pocos_datos)} brechas ({len(pocos_datos)/len(brechas)*100:.1f}%)")
print(f"  - Datos mínimos (10-49):      {len(datos_minimos)} brechas ({len(datos_minimos)/len(brechas)*100:.1f}%)")
print(f"  - Datos aceptables (50-99):   {len(datos_aceptables)} brechas ({len(datos_aceptables)/len(brechas)*100:.1f}%)")
print(f"  - Buenos datos (100+):        {len(datos_buenos)} brechas ({len(datos_buenos)/len(brechas)*100:.1f}%)")

print(f"\n⚠️ Brechas con MUY POCOS datos (<10 ejemplos):")
for bid, count in pocos_datos.items():
    brecha_name = brechas[brechas['id'] == bid]['brecha'].values[0][:60]
    print(f"  - Brecha {bid}: {count} ejemplos - {brecha_name}...")

print("\n" + "=" * 80)
print("💡 RECOMENDACIONES SEGÚN TU DATASET")
print("=" * 80)

# Calcular cuántos datos necesitarías
objetivo_minimo = 50 * len(brechas)
objetivo_bueno = 100 * len(brechas)
objetivo_excelente = 500 * len(brechas)

print(f"\n📊 Objetivo de registros totales:")
print(f"  Actual:      {len(proyectos):,} proyectos")
print(f"  Mínimo:      {objetivo_minimo:,} proyectos (50/brecha) - Necesitas {objetivo_minimo - len(proyectos):,} más")
print(f"  Bueno:       {objetivo_bueno:,} proyectos (100/brecha) - Necesitas {objetivo_bueno - len(proyectos):,} más")
print(f"  Excelente:   {objetivo_excelente:,} proyectos (500/brecha) - Necesitas {objetivo_excelente - len(proyectos):,} más")

print(f"\n🎯 Para tu caso específico:")
porcentaje_completado = (len(proyectos) / objetivo_minimo) * 100
print(f"  Tienes el {porcentaje_completado:.1f}% del objetivo mínimo")
print(f"  Faltan {objetivo_minimo - len(proyectos):,} proyectos para alcanzar el mínimo")

print("\n" + "=" * 80)

📊 ANÁLISIS DE TU DATASET

Total de proyectos: 817
Total de brechas: 48
Promedio por brecha: 17.0 proyectos

📈 Distribución de ejemplos:
  - Mínimo:  1 proyectos (brecha 1)
  - Máximo:  151 proyectos (brecha 18)
  - Mediana: 2 proyectos
  - Media:   17.4 proyectos

🔍 Categorización de brechas por cantidad de datos:
  - Muy pocos datos (<10):      31 brechas (64.6%)
  - Datos mínimos (10-49):      11 brechas (22.9%)
  - Datos aceptables (50-99):   3 brechas (6.2%)
  - Buenos datos (100+):        2 brechas (4.2%)

⚠️ Brechas con MUY POCOS datos (<10 ejemplos):
  - Brecha 1: 1 ejemplos - INDICADOR DE BRECHA POR DEFINIR...
  - Brecha 2: 3 ejemplos - PORCENTAJE DE ALIMENTOS AGROPECUARIOS DE PRODUCCIÓN Y PROCES...
  - Brecha 3: 2 ejemplos - PORCENTAJE DE CAPITALES DE DISTRITO QUE NO CUENTAN CON UN CE...
  - Brecha 4: 2 ejemplos - PORCENTAJE DE CEMENTERIOS CON CAPACIDAD INSTALADA INADECUADA...
  - Brecha 5: 1 ejemplos - PORCENTAJE DE CENTROS CUNA MÁS EN CONDICIONES INADECUADAS...
  - Brecha 6:

## ✅ ¿Es BETO la Solución Correcta para Tu Caso?

### Respuesta: **SÍ, pero con matices**

### Ventajas de BETO en tu contexto:
1. ✅ **Pre-entrenado en español**: Entiende mejor el lenguaje técnico peruano
2. ✅ **Transfer learning**: Aprovecha conocimiento previo (no partes de cero)
3. ✅ **Razonable con pocos datos**: Fine-tuning funciona con ~1000 ejemplos
4. ✅ **Estado del arte**: BERT sigue siendo competitivo en clasificación de texto

### Desventajas en tu caso:
1. ❌ **Necesita más datos** de los que tienes (~817 es muy poco)
2. ❌ **Overfitting**: Con 17 ejemplos/clase, el modelo memoriza en vez de generalizar
3. ❌ **Desbalance**: Algunas brechas tienen 150+ ejemplos, otras <10
4. ❌ **Computacionalmente costoso**: Lento para inferencia en producción

---

## 🎯 SOLUCIONES RECOMENDADAS (en orden de prioridad)

### **OPCIÓN 1: Enfoque Híbrido (LO QUE TIENES AHORA) ✅ RECOMENDADO**
**Usa embeddings como motor principal + BERT como apoyo**

```python
# Configuración recomendada para TU caso
bert_weight = 0.2  # 20% BERT, 80% embeddings
```

**Justificación:**
- Los embeddings funcionan bien con pocos datos (no necesitan entrenamiento)
- BETO aporta algo de contexto pero no es confiable aún
- Es la mejor opción con tus 817 registros actuales

**Accuracy esperado: 50-60%** 📊

---

### **OPCIÓN 2: Solo Embeddings (Más Simple) ⚡ ALTERNATIVA RÁPIDA**
**Elimina BETO completamente, usa solo búsqueda semántica**

```python
bert_weight = 0.0  # 100% embeddings
```

**Ventajas:**
- Más rápido (no necesita GPU)
- Sin entrenamiento
- Más estable (no overfitting)

**Desventajas:**
- No aprende patrones específicos de tus datos
- Depende de embeddings genéricos

**Accuracy esperado: 45-55%** 📊

---

### **OPCIÓN 3: Conseguir Más Datos (IDEAL a largo plazo) 🎯**

**Meta realista:**
- **Corto plazo (1-2 meses)**: 2,400 proyectos (50/brecha)
- **Mediano plazo (3-6 meses)**: 4,800 proyectos (100/brecha)
- **Largo plazo (1 año)**: 10,000+ proyectos

**Cómo conseguirlos:**
1. 🏛️ **Scraping de portales públicos**: SNIP, Invierte.pe, MEF
2. 🤖 **Data augmentation**: Generar variaciones de proyectos existentes
3. 👥 **Etiquetado manual**: Contratar anotadores o crowdsourcing
4. 🔄 **Active learning**: El modelo sugiere qué proyectos etiquetar

**Con 2,400+ proyectos → Accuracy esperado: 70-80%** 📊

---

### **OPCIÓN 4: Reducir Número de Brechas (Estrategia Pragmática) 🎨**

**Agrupar brechas similares para tener más datos/clase:**

Por ejemplo:
- Brecha 22, 23, 24 → **"Educación - Infraestructura Inadecuada"**
- Brecha 16, 17, 18 → **"Agua y Saneamiento"**
- Brecha 30, 31, 35 → **"Espacios Públicos"**

**De 48 clases → 15-20 macro-categorías**

**Ventajas:**
- ~40-50 ejemplos por clase (mucho mejor)
- Modelo más robusto
- Menor overfitting

**Desventajas:**
- Menor granularidad
- Puede que necesites las 48 brechas exactas

**Con agrupación → Accuracy esperado: 65-75%** 📊

---

## 🏆 RECOMENDACIÓN FINAL

### Para TU situación (817 proyectos, 48 brechas):

**1. CORTO PLAZO (ahora):**
```python
# Configuración híbrida optimizada
resultados = clasificar_hibrido_mejorado(
    texto_proyecto,
    bert_model,
    bert_tokenizer,
    embed_model,
    brecha_embeddings,
    brecha_ids,
    index,
    top_k=5,
    bert_weight=0.2,  # Poco peso a BERT (tiene pocos datos)
    use_softmax=True,
    normalize_final=True
)
```

**2. MEDIANO PLAZO (próximos 3 meses):**
- Conseguir **1,600 proyectos más** (llegar a 2,400 total)
- Reentrenar BETO con datos balanceados
- Ajustar peso: `bert_weight=0.5` (50-50)

**3. LARGO PLAZO (6-12 meses):**
- Llegar a **4,800+ proyectos** (100/brecha)
- Fine-tune del modelo de embeddings en tu dominio
- Ajustar peso: `bert_weight=0.7` (favorecer BERT)
- **Accuracy objetivo: 75-85%**

---

### ¿Vale la pena seguir con BETO?

**SÍ**, porque:
- Ya lo tienes entrenado ✅
- Aporta información útil (aunque limitada) ✅
- Es escalable cuando consigas más datos ✅

**PERO** ajusta tus expectativas:
- Con 817 registros → 50-60% accuracy máximo
- Los scores bajos (0.01-0.02) son **normales** con pocos datos
- No esperes 90%+ sin conseguir más datos

## 📈 **Recomendaciones Según Tu Situación Actual**

### **Estrategia Recomendada (Híbrida):**

#### **Fase 1: Corto Plazo (1-2 semanas)**
1. ✅ **Ajustar pesos del modelo híbrido** → Dar más peso a embeddings
   ```python
   alpha, beta = 0.2, 0.8  # 20% BERT, 80% embeddings
   ```

2. ✅ **Buscar 50-100 proyectos reales más** por cada brecha sin datos
   - Fuentes: SNIP, Invierte.pe, portales regionales
   - Enfocarte en las **brechas sin datos** (como la 48)

3. ✅ **Data augmentation moderado**
   - Generar 2-3 variaciones de proyectos existentes
   - Solo para brechas con menos de 10 ejemplos

#### **Fase 2: Mediano Plazo (1-2 meses)**
4. ✅ **Expandir dataset a 2000-5000 proyectos**
   - Mínimo 50 proyectos por brecha
   - Balancear distribución

5. ✅ **Re-entrenar con dataset completo**
   - Aumentar épocas de entrenamiento (5-10 épocas)
   - Ajustar learning rate

#### **Fase 3: Producción (3+ meses)**
6. ✅ **Dataset robusto: 10,000+ proyectos**
   - 200+ proyectos por brecha
   - Validación humana de calidad

7. ✅ **Fine-tuning avanzado**
   - Probar otros modelos (RoBERTa español, ELECTRA)
   - Ensemble de modelos

---

### **Números Concretos para Tu Caso:**

| Brecha | Min. Aceptable | Ideal | Producción |
|--------|---------------|-------|------------|
| **Total Dataset** | 1,500 proyectos | 5,000 proyectos | 10,000+ proyectos |
| **Por Brecha** | 20-30 proyectos | 100 proyectos | 200+ proyectos |
| **Brechas sin datos** | **0** (eliminar o fusionar) | **0** | **0** |

---

### **⚠️ Brechas Problemáticas en Tu Dataset:**

Según el análisis, tienes brechas **sin ningún proyecto**. Opciones:

1. **Buscar proyectos específicos** para esas brechas
2. **Fusionar brechas similares** (ej: brecha 22 y 48 son ambas educación inicial)
3. **Eliminar brechas sin datos** temporalmente del modelo

---

In [ ]:
# ====================================================================
# HERRAMIENTA 1: GENERADOR DE PARÁFRASIS CON GPT (Opcional)
# ====================================================================
# Usa esta función si tienes API key de OpenAI para aumentar tu dataset

def generar_variaciones_gpt(proyecto_original, num_variaciones=3, api_key=None):
    """
    Genera variaciones de un proyecto usando GPT para data augmentation.
    
    Args:
        proyecto_original (str): Texto del proyecto original
        num_variaciones (int): Número de paráfrasis a generar
        api_key (str): API key de OpenAI (opcional, usa variable de entorno si no se proporciona)
    
    Returns:
        list: Lista de variaciones del proyecto
    """
    try:
        from openai import OpenAI
    except ImportError:
        print("⚠️ Instala openai: pip install openai")
        return []
    
    api_key = api_key or os.getenv("OPENAI_API_KEY")
    if not api_key:
        print("⚠️ No se encontró OPENAI_API_KEY")
        return []
    
    client = OpenAI(api_key=api_key)
    
    prompt = f"""Eres un experto en proyectos de inversión pública en Perú.

Genera {num_variaciones} PARÁFRASIS del siguiente proyecto, manteniendo:
1. El mismo significado y alcance del proyecto
2. Terminología técnica apropiada para proyectos gubernamentales
3. Estructura similar a títulos de proyectos oficiales
4. La misma brecha/problema que se atiende

PROYECTO ORIGINAL:
{proyecto_original}

INSTRUCCIONES:
- Cambia palabras por sinónimos técnicos (construcción→edificación, mejoramiento→ampliación, etc.)
- Varía el orden de las frases cuando sea natural
- Mantén nombres de lugares exactos
- NO cambies el tipo de proyecto (educación sigue siendo educación)

Genera {num_variaciones} variaciones, una por línea, sin numeración ni explicaciones."""

    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.8,  # Mayor temperatura = más variedad
            max_tokens=500
        )
        
        texto = response.choices[0].message.content.strip()
        variaciones = [v.strip() for v in texto.split('\n') if v.strip()]
        
        return variaciones[:num_variaciones]
    
    except Exception as e:
        print(f"❌ Error generando variaciones: {e}")
        return []


# ====================================================================
# HERRAMIENTA 2: PARAFRASEO POR SINÓNIMOS (Sin API)
# ====================================================================

def generar_variaciones_sinonimos(proyecto_original, num_variaciones=2):
    """
    Genera variaciones reemplazando palabras clave por sinónimos.
    Método simple sin necesidad de API externa.
    
    Args:
        proyecto_original (str): Texto del proyecto original
        num_variaciones (int): Número de variaciones a generar
    
    Returns:
        list: Lista de variaciones del proyecto
    """
    # Diccionario de sinónimos comunes en proyectos públicos
    sinonimos = {
        'CONSTRUCCION': ['EDIFICACION', 'INSTALACION', 'CREACION'],
        'MEJORAMIENTO': ['AMPLIACION', 'MODERNIZACION', 'OPTIMIZACION'],
        'REHABILITACION': ['RECUPERACION', 'RESTAURACION', 'REPARACION'],
        'AMPLIACION': ['EXPANSION', 'EXTENSION', 'MEJORAMIENTO'],
        'INSTALACION': ['IMPLEMENTACION', 'ESTABLECIMIENTO', 'CREACION'],
        'AULAS': ['SALONES', 'AMBIENTES EDUCATIVOS', 'ESPACIOS EDUCATIVOS'],
        'COLEGIO': ['INSTITUCION EDUCATIVA', 'ESCUELA', 'CENTRO EDUCATIVO'],
        'PISTAS': ['VIAS', 'CALZADAS', 'CARPETA ASFALTICA'],
        'VEREDAS': ['ACERAS', 'INFRAESTRUCTURA PEATONAL', 'SENDEROS PEATONALES'],
        'AGUA POTABLE': ['SERVICIO DE AGUA', 'ABASTECIMIENTO DE AGUA', 'AGUA PARA CONSUMO'],
        'ALCANTARILLADO': ['DESAGUE', 'SANEAMIENTO', 'SISTEMA DE DESAGUE'],
    }
    
    variaciones = []
    import random
    
    for i in range(num_variaciones):
        texto_variado = proyecto_original
        
        # Reemplazar palabras clave por sinónimos
        palabras_cambiadas = 0
        for palabra_original, lista_sinonimos in sinonimos.items():
            if palabra_original in texto_variado.upper():
                # Elegir un sinónimo aleatorio
                sinonimo = random.choice(lista_sinonimos)
                
                # Reemplazar (máximo 2 cambios por variación para mantener coherencia)
                if palabras_cambiadas < 2:
                    texto_variado = texto_variado.replace(palabra_original, sinonimo)
                    texto_variado = texto_variado.replace(palabra_original.capitalize(), sinonimo.capitalize())
                    palabras_cambiadas += 1
        
        # Solo agregar si hubo cambios
        if texto_variado != proyecto_original:
            variaciones.append(texto_variado)
    
    return variaciones


# ====================================================================
# HERRAMIENTA 3: AUMENTAR DATASET COMPLETO
# ====================================================================

def aumentar_dataset(proyectos_df, brechas_col='brecha_ids', text_cols=['title', 'description'], 
                     num_variaciones=2, metodo='sinonimos', api_key=None):
    """
    Aumenta el dataset completo generando variaciones de cada proyecto.
    
    Args:
        proyectos_df (DataFrame): DataFrame original de proyectos
        brechas_col (str): Nombre de la columna con IDs de brechas
        text_cols (list): Columnas de texto a concatenar
        num_variaciones (int): Variaciones por proyecto
        metodo (str): 'gpt' o 'sinonimos'
        api_key (str): API key de OpenAI (solo para método 'gpt')
    
    Returns:
        DataFrame: Dataset aumentado con proyectos originales + variaciones
    """
    import pandas as pd
    
    nuevos_proyectos = []
    
    print(f"\n{'='*60}")
    print(f"AUMENTANDO DATASET: {len(proyectos_df)} proyectos × {num_variaciones} variaciones")
    print(f"Método: {metodo}")
    print(f"{'='*60}\n")
    
    for idx, row in tqdm(proyectos_df.iterrows(), total=len(proyectos_df), desc="Generando variaciones"):
        # Texto original
        texto_original = ". ".join([str(row[col]) for col in text_cols if pd.notna(row[col])])
        
        # Generar variaciones
        if metodo == 'gpt':
            variaciones = generar_variaciones_gpt(texto_original, num_variaciones, api_key)
        else:
            variaciones = generar_variaciones_sinonimos(texto_original, num_variaciones)
        
        # Agregar cada variación como nuevo proyecto
        for i, variacion in enumerate(variaciones):
            nuevo_proyecto = row.copy()
            nuevo_proyecto['title'] = variacion.split('.')[0]  # Primera parte como título
            nuevo_proyecto['description'] = variacion  # Completo como descripción
            nuevo_proyecto['project_id'] = f"{row['project_id']}_aug{i+1}"  # ID único
            nuevos_proyectos.append(nuevo_proyecto)
    
    # Crear DataFrame con proyectos aumentados
    df_aumentado = pd.DataFrame(nuevos_proyectos)
    
    # Combinar con dataset original
    df_completo = pd.concat([proyectos_df, df_aumentado], ignore_index=True)
    
    print(f"\n✅ Dataset aumentado:")
    print(f"  - Proyectos originales: {len(proyectos_df)}")
    print(f"  - Proyectos sintéticos: {len(df_aumentado)}")
    print(f"  - Total: {len(df_completo)}")
    
    return df_completo


# ====================================================================
# EJEMPLO DE USO
# ====================================================================

# EJEMPLO 1: Generar variaciones de un proyecto específico con sinónimos
print("\n" + "="*60)
print("EJEMPLO: Parafraseo por Sinónimos (sin API)")
print("="*60)

proyecto_ejemplo = "CONSTRUCCION DE AULAS Y MEJORAMIENTO DE VEREDAS EN COLEGIO PRIMARIO"
print(f"\nProyecto original:\n  {proyecto_ejemplo}\n")

variaciones_sin = generar_variaciones_sinonimos(proyecto_ejemplo, num_variaciones=3)
print("Variaciones generadas:")
for i, var in enumerate(variaciones_sin, 1):
    print(f"  {i}. {var}")

print("\n" + "="*60)

# EJEMPLO 2: Si tienes OpenAI API key, puedes usar GPT (descomentar para probar)
"""
print("\nEJEMPLO: Parafraseo con GPT (requiere API key)")
print("="*60)

variaciones_gpt = generar_variaciones_gpt(proyecto_ejemplo, num_variaciones=3)
if variaciones_gpt:
    print("Variaciones generadas por GPT:")
    for i, var in enumerate(variaciones_gpt, 1):
        print(f"  {i}. {var}")
"""

# EJEMPLO 3: Aumentar dataset completo (descomentar para ejecutar)
"""
# Aumentar dataset con sinónimos (2 variaciones por proyecto)
proyectos_aumentados = aumentar_dataset(
    proyectos_test, 
    num_variaciones=2, 
    metodo='sinonimos'
)

# Guardar dataset aumentado
proyectos_aumentados.to_csv("data/proyectos_aumentados.csv", index=False)
print(f"\n✅ Dataset guardado en: data/proyectos_aumentados.csv")

# IMPORTANTE: Después de generar el dataset aumentado, debes:
# 1. Ejecutar CELDA 5 nuevamente con el dataset aumentado
# 2. Ejecutar CELDA 6 nuevamente
# 3. Ejecutar CELDA 7 (re-entrenar BERT con más datos)
"""

print("\n⚠️ NOTA IMPORTANTE:")
print("El data augmentation es útil como COMPLEMENTO, no como reemplazo de datos reales.")
print("Prioriza buscar proyectos reales en bases de datos gubernamentales.")
print("="*60)


## 🔧 **Herramientas de Data Augmentation**

Si decides aumentar artificialmente tu dataset, aquí hay opciones:

### **Opción A: Parafraseo con GPT (Recomendado)**
Genera variaciones automáticas de proyectos existentes manteniendo la brecha.

### **Opción B: Traducción Inversa (Back-Translation)**
Traducir español → inglés → español para obtener variaciones naturales.

### **Opción C: Reemplazo de Sinónimos**
Cambiar palabras clave por sinónimos:
- "construcción" ↔ "edificación"
- "mejoramiento" ↔ "ampliación"
- "aulas" ↔ "salones educativos"

### **Opción D: Búsqueda de Proyectos Reales**
Fuentes gubernamentales de Perú:
- **SNIP**: Sistema Nacional de Inversión Pública
- **Invierte.pe**: Banco de proyectos
- **MEF Transparencia**: Portal de transparencia económica
- **SEACE**: Sistema Electrónico de Contrataciones

---

### **⚠️ Reglas de Oro para Data Augmentation:**

1. **Mantener coherencia semántica**: Las variaciones deben tener sentido
2. **No cambiar la brecha**: Un proyecto de educación no debe convertirse en salud
3. **Validar manualmente**: Revisar muestras de datos generados
4. **Limitar ratio**: Máximo 3-5 variaciones por proyecto original
5. **Priorizar datos reales**: Augmentation es complemento, no reemplazo

---

In [13]:
# CELDA 9 - Integración con LLM (Opcional)
# Requiere variable de entorno OPENAI_API_KEY (o adapta a otro proveedor)

import os
import json
from typing import List, Dict

try:
    from openai import OpenAI
    _openai_available = True
except ImportError:
    _openai_available = False


def build_llm_prompt(project_text: str, results: List[Dict]) -> str:
    prompt = [
        "Eres un asistente experto en clasificación de proyectos públicos según brechas.",
        "Analiza el proyecto y los candidatos (brechas) con sus puntajes combinados.",
        "Devuelve JSON con las claves: brecha_id, brecha_text, razon, confiabilidad (0-1).",
        "Si ningún candidato es adecuado, usa brecha_id=null y explica brevemente.",
        "Mantén la explicación concisa (máximo 3 frases).",
        "\nProyecto:", project_text, "\nCandidatos:"  # newline groups
    ]
    for r in results:
        prompt.append(f"- ID {r['brecha_id']}: {r['brecha_text']} | final={r['final_score']:.3f} bert={r['bert_score']:.3f} embed={r['embed_score']:.3f}")
    prompt.append("\nElige la mejor brecha y justifica.")
    return "\n".join(prompt)


def classify_with_llm(project_text: str, results: List[Dict], model: str = "gpt-4o-mini") -> Dict:
    """Usa LLM si disponible; si no, fallback por mayor final_score."""
    if not results:
        return {"brecha_id": None, "brecha_text": None, "razon": "No hay candidatos disponibles.", "confiabilidad": 0.0}

    api_key = os.getenv("OPENAI_API_KEY")
    if api_key and _openai_available:
        client = OpenAI(api_key=api_key)
        prompt = build_llm_prompt(project_text, results)
        try:
            completion = client.chat.completions.create(
                model=model,
                messages=[{"role": "system", "content": "Eres un asistente de clasificación"},
                          {"role": "user", "content": prompt}],
                temperature=0.2,
                max_tokens=400
            )
            text = completion.choices[0].message.content.strip()
            # Intentar parsear JSON si el modelo lo produjo
            if text.startswith("{"):
                try:
                    parsed = json.loads(text)
                    return {
                        "brecha_id": parsed.get("brecha_id"),
                        "brecha_text": parsed.get("brecha_text"),
                        "razon": parsed.get("razon"),
                        "confiabilidad": parsed.get("confiabilidad", 0.0),
                        "raw": text
                    }
                except json.JSONDecodeError:
                    pass
            # Si no es JSON, fallback a heurística + incluir respuesta cruda
            best = max(results, key=lambda x: x['final_score'])
            return {
                "brecha_id": best['brecha_id'],
                "brecha_text": best['brecha_text'],
                "razon": f"Selección heurística (respuesta LLM no estructurada): {text[:160]}",
                "confiabilidad": min(1.0, best['final_score']),
                "raw": text
            }
        except Exception as e:
            print(f"[LLM ERROR] {e}; usando fallback heurístico.")

    # Fallback sin API
    best = max(results, key=lambda x: x['final_score'])
    return {
        "brecha_id": best['brecha_id'],
        "brecha_text": best['brecha_text'],
        "razon": "Seleccionada por mayor puntaje híbrido (sin LLM).",
        "confiabilidad": min(1.0, best['final_score'])
    }

# Ejemplo de uso (después de ejecutar CELDA 8):
# project_text = "AMPLIACION DE 01 AULA + 01 DIRECCION EN LA CUNA JARDIN, HOSPITAL AMAZONICO- PUERTO CALLAO"
# candidates = hybrid_classify(project_text, top_k=5)
# resultado_llm = classify_with_llm(project_text, candidates)
# print(resultado_llm)


IndentationError: unexpected indent (194536428.py, line 33)